## Python 装饰器
#### 1.闭包函数
    python中，一切皆对象,这样就使得变量所拥有的属性，函数也同样拥有.这样，在函数
    内创建一个函数的行为是完全合法的。这种函数被叫做内嵌函数，这种函数只有在外部
    函数的作用域内被正常调用,在外部函数的作用域之外调用会报错。例如

In [1]:
def outFunc():
    print("1、 Hi, I am the outside function!")
    def inFunc():
        print("2、 Hi, I am the inside function!")
    inFunc()  # 只可在函数outFunc内部使用，否则会报错

outFunc()
inFunc()

Hi, I am the outside function!
Hi, I am the inside function!


NameError: name 'inFunc' is not defined

    如果内部函数里引用了外部函数里定义的对象（甚至是外层之外，但不是全局变量），
    此时内部函数就被称为闭包函数。闭包函数所应用的外部定义的变量被叫做自由变量
    闭包从语法上看非常简单，但是却有强大的作用.闭包可以将自己的代码和作用域以及
    外部函数的作用结合在一起。
##### 总结: 闭包函数 -- 函数内部定义的函数，该函数同时引用了外部变量但非全局变量 

In [3]:
def count():
    a = 1
    b = 2
    def add_in():
        c = 1
        return a + c   # a -- 自由变量
    return add_in 

res = count()
print(res)

<function count.<locals>.add_in at 0x0000019807F8F048>


##### 2. python装饰器
    python的装饰器本质上就是一个函数，它可以让其他函数在不需要做任何代码变动的
    前提下增加额外的功能，装饰器的返回值也是一个函数对象（函数的指针）。装饰器
    函数的外部函数传入要装饰的函数名字，返回经过修饰后函数的名字；内层函数（闭
    包）负责修饰被修饰函数。装饰器的几点属性：
   - 实质： 是一个函数
   - 参数： 是你要装饰的函数名（并非函数调用）
   - 返回： 是装饰完的函数名（也非函数调用）
   - 作用： 为已经存在的对象添加额外的功能
   - 特点： 不需要对对象做任何的代码上的变动
   
    python装饰器的应用场景,比如： 插入日志、性能测试、事务管理、权限校验等。装饰
    器是解决这类问题的绝佳设计。并且从引入中的列子中我们也可以归纳出：装饰器最大
    的作用就是对于我们已经写好的程序，我们可以抽离出一些雷同的代码组建多个特定功
    能的装饰器，这样我们就可以针对不同的需求去使用特定的装饰器，这时因为源码去除
    了大量泛化的内容而使得源码具有更加清晰的逻辑。

##### 2.1 函数装饰器
函数的函数装饰器: 下面以函数添加计时功能为例，讲解函数装饰器.


In [5]:
import time
def decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        func()
        end_time = time.time()
        print(end_time - start_time)
    return wrapper

@decorator
def func1():
    print("执行函数func1")
    time.sleep(3)
    
func1()

执行函数func1
3.01077938079834


   在上例中，func1是我要装饰的函数，即用装饰器显示func函数运行的时间.@decorator
   这个语法相当于执行func1=decorator(func1),为函数func1()装饰并返回。再来分析装
   饰器函数--decorator,该函数的传入参数是func(被装饰函数)，返回函数是内层函数。
   这里的内层函数--wrapper，其实就相当于闭包函数，它起到装饰给定函数的作用，
   wrapper参数为*arg,**kwargs,*arg表示参数以元组的形式传入；**kwargs表示的参数
   以字典的形式传入：

In [6]:
def wrapper(*args,**kwargs):
    print("args: ", args)
    print("kwargs: ", kwargs)
    
wrapper(["hello"], "world", 12, 34, x=56, y=78)

args:  (['hello'], 'world', 12, 34)
kwargs:  {'x': 56, 'y': 78}


##### 这里要注意的是：为了不破坏原函数的逻辑，我们要保证内层函数wrapper和被装饰函数func的传入参数和返回值类型必须保持一致。

##### 2.2 类方法的函数装饰器
  类方法的函数装饰器和函数的函数装饰器类似

In [9]:
import time

def deco(func):
    def wrapper(me_instance):
        start_time = time.time()
        func(me_instance)
        end_time = time.time()
        print(end_time - start_time)
    return wrapper

class Method(object):
    
    @deco
    def func2(self):
        print("正在执行函数func2。。。。")
        time.sleep(2)
        
m1 = Method()
m1.func2()

正在执行函数func2。。。。
2.0073139667510986


对于类方法来说，都会有一个默认的参数self，它实际表示的是类的一个实例，所以在装饰器的内部函数wrapper也要传入一个参数 - me_instance就表示将类的实例p1传给wrapper，其他的用法都和函数装饰器相同。

##### 2.2 类装饰器
  前面提到的都是让函数作为装饰器去装饰其他的函数或者方法,当然，也可以让类发挥装饰器
  的作用.函数和类在本质上没有什么不一致。只是形式可能就稍微不同

In [10]:
class Decorator():
    def __init__(self, f):
        self.f = f
        
    def __call__(self):
        print("decorator start....")
        self.f()
        print("decorator end!")
        
@Decorator
def func3():
    print("func3 is running....")

func3()

decorator start....
func3 is running....
decorator end!


这里有注意的是：__call__()是一个特殊方法，它可将一个类实例变成一个可调用对象:

In [11]:
d1 = Decorator(func3)  # d1 是类Decorator的一个实例
d1()  # 实现了__call__()方法后，d1可以被调用

decorator start....
decorator start....
func3 is running....
decorator end!
decorator end!


要使用类装饰器必须实现类中的__call__()方法，就相当于将实例变成了一个方法。

##### 2.4 装饰器链
  一个python函数也可以被多个装饰器修饰，要是有多个装饰器时，这些装饰器的执行顺序
  是如何执行的呢？

In [12]:
def makeBold(f):
    return lambda: "<b>" + f() + "</b>"

def makeItalic(f):
    return lambda:"<i>" + f() + "</i>" 

@makeBold
@makeItalic
def say():
    return "Hello!"

print(say())

<b><i>Hello!</i></b>


可见，多个装饰器的执行顺序：是从近到远依次执行。

##### 2.5 python装饰器库 - functools

In [13]:
def decorator2(func):
    def inner_func():
        pass
    return inner_func

@decorator2
def func4():
    pass

print(func4.__name__)

inner_func


上述代码最后执行的结果不是 func，而是 inner_function！这表示被装饰函数自身的信息丢
失了！怎么才能避免这种问题的发生呢？

可以借助functools.wraps()函数：

In [15]:
from functools import wraps
def decorator3(func):
    @wraps(func)
    def inner_func():
        pass
    return inner_func

@decorator3
def func5():
    pass

print(func5.__name__)

func5
